In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import glob
import seaborn as sns
from PIL import Image
import glob
import tensorflow as tf
os.environ['CUDA_VISIBLE_DEVICES']='1'

dataDir = '/data/jupyter/libin713/sample_IDCard'

import tflearn
n = 5
# Real-time data preprocessing
img_prep = tflearn.ImagePreprocessing()
img_prep.add_featurewise_zero_center(per_channel=True)

# Real-time data augmentation
img_aug = tflearn.ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_crop([600, 600], padding=4)

# Building Residual Network
net = tflearn.input_data(shape=[None, 600, 600, 3],
                         data_preprocessing=img_prep,
                         data_augmentation=img_aug)
net = tflearn.conv_2d(net, 16, 3, regularizer='L2', weight_decay=0.0001)
net = tflearn.residual_block(net, n, 16)
net = tflearn.residual_block(net, 1, 32, downsample=True)
net = tflearn.residual_block(net, n-1, 32)
net = tflearn.dropout(net, 0.5) 
net = tflearn.residual_block(net, 1, 64, downsample=True)
net = tflearn.residual_block(net, n-1, 64)
net = tflearn.batch_normalization(net)
net = tflearn.activation(net, 'relu')
net = tflearn.global_avg_pool(net)
# Regression
net = tflearn.fully_connected(net, 3, activation='softmax')
mom = tflearn.Momentum(0.001, lr_decay=0.001, decay_step=32000, staircase=True)
net = tflearn.regression(net, optimizer=mom,
                         loss='categorical_crossentropy')
# Training
model = tflearn.DNN(net, checkpoint_path='resnet_Idcard.model',
                    max_checkpoints=10, tensorboard_verbose=0,
                    clip_gradients=0.)

model.load("resnet_Idcard.model")

INFO:tensorflow:Restoring parameters from /data/jupyter/libin713/sample_IDCard/VGG/resnet_Idcard.model


In [21]:
from PIL import Image
#img_path = os.listdir('/data/jupyter/libin713/sample_IDCard/test1')[0:500]
img_path = ['IMG_0342.JPG']
#img_path = ['test1.jpg','test2.jpg','test3.jpg','test4.jpg',]
for i in img_path:
    #img = Image.open('/data/jupyter/libin713/sample_IDCard/test/'+i)
    img = Image.open(i)
    img = img.resize((600,600))
    img = np.asarray(img,dtype = "float32")
    img = img.reshape([-1,600,600,3])
    print(model.predict(img))

[[0. 1. 0.]]


In [3]:
print(img.shape)
img = img.reshape([-1,600,600,3])
print(img.shape)

(600, 600, 3)
(1, 600, 600, 3)


In [4]:
model.predict(img)

array([[0., 0., 1.]], dtype=float32)